Imports and Set up
===

In [21]:
import pymc3 as pm
import theano.tensor as T
from numpy import random, sum as nsum, concatenate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import theano

In [31]:
print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.8


In [22]:
# Set up for a potential error
from __future__ import absolute_import
from __future__ import print_function
from __future__ import unicode_literals

In [23]:
%matplotlib inline

In [24]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

In [25]:
plt.rcParams['figure.figsize'] = (15, 10)

plt.rcParams['font.size'] = 16

Linear Regression
===

In [26]:
# Set up basic parameters
num_features = 10
num_observed = 1000

In [27]:
# Choose random values for the actual alpha and betas
alpha_a = random.normal(size=1)

betas_a = random.normal(size = num_features)

# Create fake predictor data
X_train = random.normal(size=(num_observed, num_features))

In [28]:
# Calculate the actual data, but put a bit of noise in
y_a = alpha_a + nsum(betas_a[None,:] * X_train, 1) + random.normal(size=(num_observed))

### Strings vs Unicode:

### Tip #1: Use the appropriate string type for your Python version.

In [29]:
# Set up the PyMC model
lin_reg_model = pm.Model()
with lin_reg_model:
    
    # CHANGE: All names are now cast as strings.
    alpha = pm.Normal('alpha', mu=0, tau=10.**-2, shape=(1))
    betas = pm.Normal('betas', mu=0, tau=10. ** -2, shape=(1, num_features))
    
    s = pm.HalfNormal('s', tau=1)
    
    temp = alpha + T.dot(betas, X_train.T)

    y = pm.Normal('y', mu=temp , tau=s ** -2, observed=y_a)

In [30]:
# Sample from the model
with lin_reg_model:
    step = pm.NUTS()
    nuts_trace = pm.sample(2e3, step)

Multiprocess sampling (4 chains in 4 jobs)
NUTS: [s, betas, alpha]


TypeError: 'float' object cannot be interpreted as an integer

In [ ]:
# Plot the trace
# Note: You can specify a burn-in period by indexing appropriately
pm.traceplot(nuts_trace[1000:])

In [ ]:
# Look at a summary of the trace
pm.summary(nuts_trace[1000:])

In [ ]:
alpha_a

In [ ]:
betas_a

Linear Regression: Now with Data!
===

In [ ]:
from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import r2_score

In [ ]:
boston = load_boston()

In [ ]:
boston

In [ ]:
boston_features = pd.DataFrame(boston.data)

In [ ]:
boston_features.columns = boston.feature_names

In [ ]:
boston_features.CHAS = boston_features.CHAS.astype('bool')

In [ ]:
boston_features.RAD = boston_features.RAD.astype('int')

In [ ]:
boston_target = pd.DataFrame(boston.target)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    boston_features, boston_target, test_size=0.3)

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, Y_train)

In [ ]:
lr.score(X_test, Y_test)

### DataFrames don't work!

### Tip #2: Turn Dataframes into numpy Arrays

### Booleans don't work!

### Tip #3: Cast Booleans as ints

In [ ]:
# CHANGE: Boolean column is cast as an integer
X_train.CHAS = X_train.CHAS.astype(int)

In [ ]:
model_input = theano.shared(np.array(X_train))
# CHANGE: Y_train is cast as a numpy array
model_output = theano.shared(np.array(Y_train))

In [ ]:
# Set up the PyMC model
lin_reg_model_w_data = pm.Model()
with lin_reg_model_w_data:
    
    alpha = pm.Normal(b'alpha', mu=0, tau=10.**-2, shape=(1))
    betas = pm.Normal(b'betas', mu=0, tau=10. ** -2, shape=(1, len(X_test.columns)))

    s = pm.HalfNormal(b's', tau=1)
    
    temp = alpha + T.dot(model_input, betas.T)

    y = pm.Normal(b'y', temp , tau=s ** -2, observed=model_output)

In [ ]:
# Note: If you don't specify a sampler, PyMC3 will choose one for you
with lin_reg_model_w_data:
    nuts_trace = pm.sample(8e3)

In [ ]:
pm.traceplot(nuts_trace[5000:])

In [ ]:
pm.summary(nuts_trace[5000:])

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

Now we can score our model
===

In [ ]:
X_test.CHAS = X_test.CHAS.astype(int)

In [ ]:
# Replace shared variables with testing set
model_input.set_value(np.array(X_test))
model_output.set_value(np.array(Y_test))

In [ ]:
# Create posterior predictive samples
ppc = pm.sample_ppc(nuts_trace[1000:], model=lin_reg_model_w_data, samples=1000)

In [ ]:
pred = ppc['y'].mean(axis=0)

In [ ]:
r2_score(Y_test, pred)

Hiearchical Linear Regression
===

In [ ]:
# Set up basic parameters
num_cats = 4
num_per_cat = 50
num_observed = num_per_cat * num_cats
num_features = 3

In [ ]:
# Set up categories
cat = concatenate([[i] * num_per_cat for i in range(num_cats)])

# Simulate the features
features = random.normal(size=(num_observed, num_features))

In [ ]:
# Choose random values to represent the actual alphas and betas
alpha_a = random.normal(size=(num_cats))
beta_a = random.normal(size=(num_cats, num_features))

In [ ]:
# Calculate the actual data, but put a bit of noise in
y = alpha_a[cat] + nsum(beta_a[cat] * features, 1) + random.normal(size=(num_observed))

In [ ]:
# Set up the PyMC model
hlm = pm.Model()
with hlm:
    
    # Both alpha and beta are drawn from similar distributions
    mu_alpha = pm.Normal(b"mu_alpha", mu=0, sd=10, shape=(1))
    sigma_alpha = pm.Uniform(b"sigma_alpha", .0, 10, testval=2.)
    
    mu_beta = pm.Normal(b"mu_beta", mu=0, sd=10, shape=(1))
    sigma_beta = pm.Uniform(b"sigma_beta", .0, 10, testval=2.)
    
    # Simulate the alphas
    alpha = pm.Normal(b"alpha", mu_alpha, sigma_alpha, shape=(num_cats))
    
    # Simulate the betas
    beta = pm.Normal(b'beta', mu_beta, sigma_beta, shape=(num_cats, num_features))
    
    c = T.constant(cat)
    
    # Simulate the noise
    s = pm.Uniform(b"s", .01, 10, shape=num_cats)

    yd = pm.Normal(b'y', mu=alpha[c] + T.sum(beta[c]*features, 1), tau=s[c] ** -2, observed=y)

    step = pm.NUTS()

In [ ]:
# Actually sample the model
# Note: you can do this either via the "with" statement or by specifying the model
tr = pm.sample(3e4, step, model=hlm)

In [ ]:
# Plot the variables
pm.traceplot(tr)

In [ ]:
# See a summary
pm.summary(tr)

In [ ]:
alpha_a

In [ ]:
beta_a

Logistic Regression
===

In [ ]:
def numpy_invlogit(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
# Set up basic parameters
num_features = 10
num_observed = 1000

In [ ]:
# Choose random values for the actual alpha and betas
alpha_a = random.normal(size=1)

betas_a = random.normal(size = num_features)

# Create fake predictor data
X_train = random.normal(size=(num_observed, num_features))
X_test =  random.normal(size=(num_observed, num_features))

In [ ]:
# Calculate the outcomes
Y_train = random.binomial(1, numpy_invlogit(alpha_a + np.sum(betas_a[None, :] * X_train, 1)))
Y_test = random.binomial(1, numpy_invlogit(alpha_a + np.sum(betas_a[None, :] * X_test, 1)))

In [ ]:
model_input = theano.shared(X_train)
model_output = theano.shared(Y_train)

In [ ]:
log_reg_model = pm.Model()

with log_reg_model:
    alpha = pm.Normal(b'alpha', mu=0, tau=10.**-2, shape=(1))
    betas = pm.Normal(b'betas', mu=0, tau=10. ** -2, shape=(1, num_features))
    
    p = pm.invlogit(alpha + T.sum(betas*model_input, 1))

    o = pm.Bernoulli(b'o', p, observed=model_output)

### Tip #4: Make sure there are no NaNs or infinities in your data or you'll get weird results with ADVI

Let's use ADVI
===

In [ ]:
with log_reg_model:
    v_params = pm.variational.advi(n=10000)

In [ ]:
plt.plot(v_params.elbo_vals)

In [ ]:
with log_reg_model:
    advi_trace = pm.variational.sample_vp(v_params, draws=5000)

In [ ]:
with log_reg_model:
    
    step = pm.NUTS(scaling=v_params.stds)

    nuts_trace = pm.sample(8e3, step, start=v_params.means)


In [ ]:
pm.traceplot(nuts_trace[1000:])

In [ ]:
pm.summary(nuts_trace[1000:])

In [ ]:
# Replace shared variables with testing set
model_input.set_value(X_test)
model_output.set_value(Y_test)

In [ ]:
# Create posterior predictive samples from ADVI
ppc = pm.sample_ppc(advi_trace, model=log_reg_model, samples=1000)

# Use probability of > 0.5 to assume prediction of class 1
pred = ppc['o'].mean(axis=0) > 0.5

In [ ]:
print('ADVI Accuracy = {}%'.format((Y_test == pred).mean() * 100))

In [ ]:
# Create posterior predictive samples from NUTS
ppc = pm.sample_ppc(nuts_trace[1000:], model=log_reg_model, samples=1000)

# Use probability of > 0.5 to assume prediction of class 1
pred = ppc['o'].mean(axis=0) > 0.5

In [ ]:
print('NUTS Accuracy = {}%'.format((Y_test == pred).mean() * 100))

API-ify your model
===

### Note: you can save your trace and reload it.

In [ ]:
# import pickle
# fileObject = open("advi_trace.pickle",'w')  
#pickle.dump(advi_trace, fileObject)
#fileObject.close()

In [ ]:
# advi_trace = pickle.load(open("pickle_jar/advi_trace.pickle",'r')  )

In [ ]:
# First, create some test data
API_test =  random.normal(size=(1, num_features))

In [ ]:
API_Y_test = random.binomial(1, numpy_invlogit(alpha_a + np.sum(betas_a[None, :] * API_test, 1)))

In [ ]:
API_Y_test

In [ ]:
model_input = theano.shared(API_test)

### Tip #5: if you train your model with observed data, you need to put in fake observed data to sample from it

In [ ]:
# Put in some fake data
API_fake_Y = 0

In [ ]:
API_model = pm.Model()

with API_model:
    alpha = pm.Normal(b'alpha', mu=0, tau=2.**-2, shape=(1))
    betas = pm.Normal(b'betas', mu=0, tau=2. ** -2, shape=(1, num_features))
    
    p = pm.invlogit(alpha + T.sum(betas*model_input, 1))

    # CHANGE: The outcome is now set with an observed value of fake data
    o = pm.Bernoulli(b'o', p, observed=API_fake_Y)


In [ ]:
# Create posterior predictive samples
ppc = pm.sample_ppc(advi_trace, model=API_model, samples=1000)

In [ ]:
# Use probability of > 0.5 to assume prediction of class 1
pred = ppc['o'].mean(axis=0) > 0.5

In [ ]:
pred

Hiearchical Logistic Regression
===

In [ ]:
# Set up basic parameters
num_cats = 4

# Need lots of data to converge
num_per_cat = 15000
num_observed = num_per_cat * num_cats
num_features = 100

In [ ]:
# Set up categories
cat = concatenate([[i] * num_per_cat for i in range(num_cats)])

In [ ]:
# Simulate the features
features = np.random.normal(size=(num_observed, num_features))

In [ ]:
alpha_a = np.random.normal(size=(num_cats))
beta_a = np.random.normal(size=(num_cats, num_features))

In [ ]:
# Calculate the actual data
p = alpha_a[cat] + nsum(beta_a[cat] * features, 1)

In [ ]:
outcomes = np.random.binomial(1, numpy_invlogit(p))

In [ ]:
hlr = pm.Model()

with hlr:
    # Both alpha and beta are drawn for the same distributions
    mu_alpha = pm.Normal(b"mu_alpha", mu=0, sd=10, shape=(1))
    sigma_alpha = pm.Uniform(b"sigma_alpha", .0, 10, testval=2.)
    
    mu_beta = pm.Normal(b"mu_beta", mu=0, sd=10, shape=(1))
    sigma_beta = pm.Uniform(b"sigma_beta", 0, 10, testval=2.)
    
    alpha = pm.Normal(b'alpha', mu=mu_alpha, tau=sigma_alpha, shape=(num_cats))
    beta = pm.Normal(b'beta', mu=mu_beta, tau=sigma_beta, shape=(num_cats, num_features))
    
    c = T.constant(cat)

    p = pm.invlogit(alpha[c] + T.sum(beta[c]*features, 1))
    
    o = pm.Bernoulli(b'o', p, observed=outcomes)

In [ ]:
with hlr:
    v_params = pm.variational.advi(n=10000)

In [ ]:
plt.plot(v_params.elbo_vals)

In [ ]:
with hlr:
    
    step = pm.NUTS(scaling=v_params.stds)
    trace = pm.sample(20000, step, start=v_params.means)

### Tip #6: You can stop your trace part way through and still use it!

In [ ]:
pm.traceplot(trace)

In [ ]:
pm.summary(trace)

In [ ]:
beta_a

In [ ]:
alpha_a

In [ ]:
pm.forestplot(trace, varnames=['mu_alpha', 'mu_beta', 'alpha', 'sigma_alpha', 'sigma_beta'])